# Performance of Models: Old and New

![Old BOA](../plots/notebook_plots/model_goodness_of_fit/old_boa_all_models.png "Old BOA")
![New BOA](../plots/notebook_plots/model_goodness_of_fit/new_boa_all_models.png "New BOA")

# Comparison of Performance of Individual Models: Old vs New
![GP vs GP](../plots/notebook_plots/model_goodness_of_fit/comparison_old_gp_new_gp.png "GP vs GP")

![GPAR vs GPAR](../plots/notebook_plots/model_goodness_of_fit/comparison_old_gpar_new_gpar.png "GPAR vs GPAR")

![MF-GPAR-5](../plots/notebook_plots/model_goodness_of_fit/comparison_old_mf_gpar_5_new_mf_gpar_5.png "MF-GPAR-5")